# Deploy an MLflow model with SageMaker

## Install MLflow

In [ ]:
!pip install -q mlflow==2.0.1

## Setup environment

In [1]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
from sklearn.datasets import fetch_california_housing
from mlflow.deployments import get_deploy_client

In [2]:
# name of the AWS region to which to deploy the application
region = sagemaker.Session().boto_region_name
# we are using the notebook instance role for training in this example
role = 'arn:aws:iam::118756721574:role/sm_execution'
# role = sagemaker.get_execution_role()
# uri of your remote mlflow server
tracking_uri = 'http://13.40.213.205:5000'
# set remote mlflow server
mlflow.set_tracking_uri(tracking_uri)

## Build MLflow docker image to serve the model with SageMaker 

In [6]:
!mlflow sagemaker build-and-push-container

Login Succeeded

2023/05/08 17:42:02 INFO mlflow.models.docker_utils: Building docker image with name mlflow-pyfunc
#1 [internal] load build definition from Dockerfile
#1 sha256:5b0acfd904d1785757e180ed2e7f69c0b77856625903a64a9499c0db33b48a6e
#1 transferring dockerfile: 30B 0.0s
#1 transferring dockerfile: 2.40kB 0.1s done
#1 DONE 0.1s

#2 [internal] load .dockerignore
#2 sha256:9ac6bb66441be6c691733f1853b5589aa9f9aca5fe3875e769017cd2ed51fa31
#2 transferring context: 2B 0.0s done
#2 DONE 0.1s

#3 [internal] load metadata for docker.io/library/ubuntu:20.04
#3 sha256:8e67b796a66f85f06793e026943184e32d365c77929e94d2ac98b34a1e1cb30e
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 sha256:02c6f2784275c1ed335abe8afcdebdc2eb588e51a3a2cb640936ade05fd2236d
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:20.04
#3 sha256:8e67b796a66f85f06793e026943184e32d365c77929e94d2ac98b34a1e1cb30e
#3 DONE 2.3s

#5 [ 1/20] FROM docker.io/library/ubuntu:20.04@sha256:db8bf6f4fb351aa7


The push refers to repository [118756721574.dkr.ecr.eu-west-2.amazonaws.com/mlflow-pyfunc]
8eb826a92851: Preparing
e99700231aa9: Preparing
3048945286ee: Preparing
372a9969cb16: Preparing
3280f248c98d: Preparing
14e4b062897a: Preparing
9f1b89b6c512: Preparing
e6ceb1f2e72b: Preparing
6d9f79e08353: Preparing
14e4b062897a: Waiting
9f1b89b6c512: Waiting
e6ceb1f2e72b: Waiting
6d9f79e08353: Waiting
e25f491d4d07: Preparing
d4e4bcc6600a: Preparing
ac727932f501: Preparing
91e467465f83: Preparing
d4e4bcc6600a: Waiting
e25f491d4d07: Waiting
ac727932f501: Waiting
f07221d83cb4: Preparing
91e467465f83: Waiting
0477ec30c82a: Preparing
5e099fd6def3: Preparing
5d27dc224e7d: Preparing
f07221d83cb4: Waiting
0477ec30c82a: Waiting
5e099fd6def3: Waiting
7f4c37006064: Preparing
f870ae874da9: Preparing
5d27dc224e7d: Waiting
6f37ca73c74f: Preparing
7f4c37006064: Waiting
6f37ca73c74f: Waiting
f870ae874da9: Waiting
372a9969cb16: Pushed
8eb826a92851: Pushed
3280f248c98d: Pushed
e6ceb1f2e72b: Pushed
14e4b062897a: 

In [7]:
# URL of the ECR-hosted Docker image the model should be deployed into
image_uri = '118756721574.dkr.ecr.eu-west-2.amazonaws.com/mlflow-pyfunc:2.3.1'

## Deploy a SageMaker endpoint with our scikit-learn model

In [8]:
endpoint_name = 'boston-housing'
# The location, in URI format, of the MLflow model to deploy to SageMaker.
model_uri = 'models:/boston-housing/1'

In [9]:
config={
    'execution_role_arn': role,
    'image_url': image_uri,
    'instance_type': 'ml.m5.xlarge',
    'instance_count': 1, 
    'region_name': region
}

client = get_deploy_client("sagemaker")

client.create_deployment(
    name=endpoint_name,
    model_uri=model_uri,
    flavor='python_function',
    config=config
)

2023/05/08 17:59:48 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2023/05/08 17:59:48 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2023/05/08 17:59:50 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-eu-west-2-118756721574` not found. Creating...
2023/05/08 17:59:52 INFO mlflow.sagemaker: Bucket creation response: {'ResponseMetadata': {'RequestId': '23HCBFVPXRANBCMW', 'HostId': 'w45PKfvcJpl9gztqmw+U9DUfwa/25y15VDURxQrPXmwkRlx0r/YNKQP3MFN+xs+hSWHLe1zgMBs=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'w45PKfvcJpl9gztqmw+U9DUfwa/25y15VDURxQrPXmwkRlx0r/YNKQP3MFN+xs+hSWHLe1zgMBs=', 'x-amz-request-id': '23HCBFVPXRANBCMW', 'date': 'Mon, 08 May 2023 16:59:52 GMT', 'location': 'http://mlflow-sagemaker-eu-west-2-118756721574.s3.amazonaws.com/', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'Location': 'http://mlflow-sagemaker-eu-west-2-118756721574.s3.amazonaws.com/'}
2023/05/08 18:02:20 INFO mlflo

{'name': 'boston-housing', 'flavor': 'python_function'}

## Predict

In [10]:
# load boston dataset
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)

In [11]:
client = get_deploy_client(f"sagemaker:/{region}")

payload = df.iloc[[0]]
prediction = client.predict(endpoint_name, df.iloc[[0]])

print(f'Payload: {payload}')
print(f'Prediction: {prediction}')

Payload:    MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude   
0  8.3252      41.0  6.984127    1.02381       322.0  2.555556     37.88  \

   Longitude  
0    -122.23  
Prediction: {'predictions': [4.2108459375]}


## Delete endpoint

In [12]:
client.delete_deployment(endpoint_name, config=config)

2023/05/08 18:07:44 INFO mlflow.sagemaker: Deleted endpoint with arn: arn:aws:sagemaker:eu-west-2:118756721574:endpoint/boston-housing
2023/05/08 18:07:44 INFO mlflow.sagemaker: Waiting for the delete operation to complete...
2023/05/08 18:07:44 INFO mlflow.sagemaker: Deletion is still in progress. Current endpoint status: Deleting
2023/05/08 18:07:49 INFO mlflow.sagemaker: The deletion operation completed successfully with message: "The SageMaker endpoint was deleted successfully."
2023/05/08 18:07:49 INFO mlflow.sagemaker: Cleaning up unused resources...
2023/05/08 18:07:49 INFO mlflow.sagemaker: Deleted associated endpoint configuration with arn: arn:aws:sagemaker:eu-west-2:118756721574:endpoint-config/boston-housing-config-zq2faugzq3o2-dnqxa73nw
2023/05/08 18:07:50 INFO mlflow.sagemaker: Deleted associated model with arn: arn:aws:sagemaker:eu-west-2:118756721574:model/boston-housing-model-48l8jkhaqcqgwqkknrs3ca
